<a href="https://colab.research.google.com/github/LCSMalves/bot-para-monitoria/blob/main/monitorias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-05-20')

In [ ]:
try:
    df = pd.read_csv('conversas - Página1.csv')
except FileNotFoundError:
    print("Erro: O arquivo 'conversas.csv' não foi encontrado. Por favor, verifique o caminho do arquivo.")
    data = {'Conversa': ["Atendimento bom, cliente satisfeito.", "Atendimento ruim, cliente insatisfeito.", "Atendimento ok, sem problemas.", "Atendimento excelente, cliente muito feliz."],
            'Agente': ["Agente A", "Agente B", "Agente A", "Agente C"]}
    df = pd.DataFrame(data)
    print("Criado um DataFrame de exemplo para continuar.")


if 'Avaliação Gemini' not in df.columns:
    df['Avaliação Gemini'] = None

display(df.head())

In [ ]:

checklist = """

Informe se a execução foi correta ou incorreta;
Quem foi o responsavel pela FCR, o Passageiro, o Agente ou os processos da Latam;
Se o procedimento solicitado pelo passageiro foi realizado ou não realizado;

E selecione um desses motivos para casos não resolvidos:
VALORES ALTOS
PROCEDIMENTO INCORRETO
PENDENTE INDEVIDO
NÃO RESOLVIDO
CLIENTE NÃO RESPONDE
POSTURA
SINERGIA
VOO INDISPONIVEL
RESOLVIDO INDEVIDO
REGRA DO PRODUTO
SEM MOTIVO
FOI REALIZADO
DEMORA DO ATENDIMENTO
CORTE DE ATENDIMENTO
TRANSFERENCIA HVC
ERRO CART.PASS
FALHA NA FERRAMENTA
TELEFONE NÃO ATENDE
COMPRA AGENCIA
PAX NÃO ACEITA VALORES
check-in não permitido
TRANSFERENCIA INDEVIDA
CLIENTE EM DOIS CANAIS DE ATENDIMENTO;

"""

for index, row in df.iterrows():
    conversa = row['Conversa']

    prompt = f"""

    haja como um monitor de qualidade da companhia aerea Latam e avalie a seguinte conversa de chat do ponto de vista de qualidade de atendimento, separe cada agente que participou do contato,
    Leve em conta as regras da companhia aérea, seguindo esses requisitos:(aponte abaixo caso um deles não seja cumprido)

    Assistente não pode direcionar o pax para o aeroporto;
    Assistente não pode direcionar para outro assistente;
    Em remarcações voluntarias, não podemos mudar de mercado, exemplo, de nacional para internacional;
    Seguir as regras comerciais da Latam;
    Não seguimos a CDC e sim a lei ANAC;
    Não abrimos casos de reclamação ou direcionamos o pax para abertura no site, resolvemos em linha a situação já passando a resolução no atendimento;
    Caso o passageiro fique mais de 3 minutos sem responder, o assistente deve questionar se o passageiro ainda esta na online, caso passe mais 2 minutos sem resposta, o assistente deve enviar para pendente (Você ainda está disponível para continuar com a nossa conversa? Se desejar continuar, responda a esta mensagem e teremos o prazer de transferi-lo para um de nossos executivos.) e não para a pesquisa de satisfação;
    Reservas vendidas por agencia, devemos realizar apenas remarcação dentro de 48 horas antes do voo e em casos de remarcação por parte da companhia e check-in, não realizamos nenhum outro tipo de atendimento para agencias;

    Após isso realize o seguinte:
    Primeiramente, apresente os dados do contato;
    Apresente o tempo de resposta do agente durante todo o contato, avaliando esse tempo (considere auto casos acima de 5 minutos), mas caso agente utilize o script de pausa longa, esse tempo pode ser maior (10 minutos);
    Verifique a ortografia e gramática do texto;
    Em seguida apresente uma avaliação do contato;
    Apresente os pontos positivos e negativos do atendimento;
    Apresente um feedback para ser enviado via chat para o agente;

    Não avaliar o Bot;

    Por favor, seguindo o checklist abaixo, explicando brevemente o porquê:

    Checklist:
    {checklist}

    Conversa:
    {conversa}

    Avaliação:
    """

    try:
        response = gemini_model.generate_content(prompt)

        df.loc[index, 'Avaliação Gemini'] = response.text

    except Exception as e:
        print(f"Erro ao processar a conversa {index}: {e}")
        df.loc[index, 'Avaliação Gemini'] = f"Erro: {e}"

display(df.head())

In [ ]:
output_filename = 'conversas_avaliadas.csv'
df.to_csv(output_filename, index=False)

print(f"Planilha atualizada salva como '{output_filename}'")

In [ ]:
print("Amostra das avaliações por agente:")

for agente, avaliacoes in df.groupby('Agente')['Avaliação Gemini']:
    print(f"\nAgente: {agente}")
    for i, avaliacao in enumerate(avaliacoes.dropna().tolist()):
        if i >= 5:
            break
        print(f"- {avaliacao[:200]}...")